In [31]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import requests
from math import radians, sin, cos, sqrt, atan2
import re
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, FunctionMessage
from langchain_core.runnables import RunnablePassthrough, RunnableMap, Runnable
from langchain_core.messages import FunctionMessage

## 데이터 불러오기, function calling 함수 정의

In [10]:
# 대피소 정보 불러오기
with open("../SourceCode/shelters.json", "r", encoding="utf-8") as f:
    shelters = json.load(f)

In [11]:
# shelters 리스트를 데이터프레임으로 변환
df = pd.DataFrame(shelters)
df = df[df['SHNT_PSBLTY_NOPE'] != '0']  # 대피가능인원수 0인 곳을 제외

In [12]:
print(df.head())

  GRND_UDGD_SE ORTM_UTLZ_TYPE   SGG_CD     FCLT_CD    FCLT_SE_CD  \
0            1            주차장  3910000  S202400002  3              
1            1            주차장  3910000  S202400001  3              
2            1            주차장  4020000  S202400001  3              
3            1          지하주차장  3650000  S201700010  3              
4            1          지하주차장  3650000  S201700011  3              

     ROAD_NM_CD  LOT_MIN SHNT_PSBLTY_NOPE  FCLT_SCL  LOT_SEC  ... LAT_PROVIN  \
0  3352297             6            34761     28678       24  ...         37   
1  3352295             6            37152     30651       27  ...         37   
2  3182030            56            39997     32998       42  ...         37   
3  3165028            23             6577      5426       55  ...         36   
4  3165028            23             5126      4229       56  ...         36   

  LAT_MIN MNG_INST_TELNO                     FCLT_NM  FCLT_DSGN_DAY  \
0       1  031-8024-4900  평택지제역동문굿모닝힐맘시

In [13]:
# DMS를 소수점 좌표로 변환하는 함수
def dms_to_dd(degrees, minutes, seconds):
    return degrees + minutes / 60 + seconds / 3600

# 위도 변환 (LAT_PROVIN, LAT_MIN, LAT_SEC를 사용)
df['위도'] = df.apply(lambda row: dms_to_dd(
    float(row['LAT_PROVIN']), 
    float(row['LAT_MIN']), 
    float(row['LAT_SEC'])
), axis=1)

# 경도 변환 (LOT_PROVIN, LOT_MIN, LOT_SEC를 사용)
df['경도'] = df.apply(lambda row: dms_to_dd(
    float(row['LOT_PROVIN']), 
    float(row['LOT_MIN']), 
    float(row['LOT_SEC'])
), axis=1)

# 주소 컬럼 설정 (도로명 주소 사용)
df['주소'] = df['FCLT_ADDR_RONA']
# 장소 컬럼 설정
df['시설명'] = df['FCLT_NM']

# 필요한 컬럼만 선택
df_result = df[['시설명','주소', '위도', '경도']]

print(df_result.head())

                          시설명                                        주소  \
0  평택지제역동문굿모닝힐맘시티4단지 지하주차장 1층  경기도 평택시 신촌5로 56 (칠원동, 평택지제역동문굿모닝힐맘시티4단지)   
1  평택지제역동문굿모닝힐맘시티2단지 지하주차장 1층  경기도 평택시 신촌3로 12 (칠원동, 평택지제역동문굿모닝힐맘시티2단지)   
2             힐스테이트 금정역 지하주차장         경기도 군포시 엘에스로 143 (금정동, 힐스테이트 금정역)   
3                유등마을아파트 104동           대전광역시 중구 수침로 138 (태평동, 유등마을아파트)   
4                유등마을아파트 106동           대전광역시 중구 수침로 138 (태평동, 유등마을아파트)   

          위도          경도  
0  37.025278  127.106667  
1  37.026389  127.107500  
2  37.373333  126.945000  
3  36.332222  127.398611  
4  36.331667  127.398889  


In [14]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    위도와 경도를 받아 두 지점 사이의 거리를 킬로미터 단위로 계산하는 함수
    """
    # 위도와 경도를 라디안으로 변환
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)

    # 위도와 경도의 차이 계산
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine 공식 적용
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6371  # 지구 반지름 (킬로미터 단위)
    distance = R * c
    return distance

### 카카오맵 API 발급 받으셔야 합니다. 주소 : https://apis.map.kakao.com/ 

발급받고 .env 파일에 REST_API_KEY="API key 입력하세요"

In [15]:
load_dotenv()
kakaoapikey = os.getenv("REST_API_KEY") # 카카오 API 호출

def get_coordinates(query):
    """
    카카오맵 API를 이용해 검색 질의어(query)의 경도와 위도를 반환하는 함수.
    
    Args:
        query (str): 검색할 장소 또는 주소.

    Returns:
        tuple: (longitude, latitude) - 경도(x), 위도(y)
    """
    # 카카오맵 API 엔드포인트
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    
    # 요청 헤더
    headers = {
        "Authorization": f"KakaoAK {kakaoapikey}"
    }
    
    # 요청 파라미터
    params = {
        "query": query,
        "size": 1  # 결과를 하나만 가져오도록 설정
    }
    
    # API 요청
    response = requests.get(url, headers=headers, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()  # JSON 응답 파싱
        documents = data.get("documents", [])
        if documents:
            # 첫 번째 검색 결과의 경도(x)와 위도(y) 반환
            x = documents[0].get("x")
            y = documents[0].get("y")
            return (x, y)
        else:
            print("검색 결과가 없습니다.")
            return None
    else:
        # 에러 메시지 출력
        print(f"API 요청 실패: {response.status_code}, {response.text}")
        return None

## API 호출 테스트 입니다.

In [16]:
''' 키워드 검색시 주소를 호출하지 못함 -> 키워드 검색 로직 추가 '''
load_dotenv()
kakaoapikey = os.getenv("REST_API_KEY") # 카카오 API 호출
def get_coordinates(query):
    """
    검색 질의어(query)를 기반으로 주소 검색 API와 장소 검색 API를 순차적으로 호출하여 경도와 위도를 반환.

    Args:
        query (str): 검색할 주소 또는 장소.
        kakaoapikey (str): 카카오맵 REST API 키.

    Returns:
        tuple: (longitude, latitude) - 경도(x), 위도(y)
    """
    # 1. 주소 검색 API 호출
    address_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakaoapikey}"}
    params = {"query": query, "size": 1}

    address_response = requests.get(address_url, headers=headers, params=params)
    if address_response.status_code == 200:
        data = address_response.json()
        documents = data.get("documents", [])
        if documents:
            x = documents[0].get("x")
            y = documents[0].get("y")
            return (x, y)
    
    # 2. 장소 검색 API 호출 (주소 검색 결과가 없을 경우)
    keyword_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    keyword_response = requests.get(keyword_url, headers=headers, params=params)
    if keyword_response.status_code == 200:
        data = keyword_response.json()
        documents = data.get("documents", [])
        if documents:
            x = documents[0].get("x")
            y = documents[0].get("y")
            return (x, y)
    
    # 결과 없음
    print("검색 결과가 없습니다.")
    return None


# 테스트 실행
if __name__ == "__main__":
    query = "동대구역"  # 검색할 장소 또는 주소
    coordinates = get_coordinates(query)
    if coordinates:
        print(f"경도: {coordinates[0]}, 위도: {coordinates[1]}")

경도: 128.628393775388, 위도: 35.8793239931795


In [17]:
def find_nearest_shelters(address: str) -> str:
    """
    주어진 주소를 기반으로 가장 가까운 대피소 정보를 반환하는 함수.
    """
    coordinates = get_coordinates(address)
    if coordinates:
        user_lon = float(coordinates[0])
        user_lat = float(coordinates[1])

        
        def calculate_distance(row):
            try:
                shelter_lat = float(row['위도'])
                shelter_lon = float(row['경도'])
                return haversine_distance(user_lat, user_lon, shelter_lat, shelter_lon)
            except (ValueError, TypeError):
                return None

        # 거리 계산 및 데이터프레임 업데이트
        df_result['거리'] = df_result.apply(calculate_distance, axis=1)
        df_valid = df_result[df_result['거리'].notnull()]
        df_sorted = df_valid.sort_values(by='거리')
        df_top3 = df_sorted.head(3)

        # 결과 문자열 생성
        result = "\n가장 가까운 대피소 정보 (거리순):"
        for idx, row in df_top3.iterrows():
            result += f"\n\n[{idx+1}]"
            result += f"\n시설명: {row['시설명']}"
            result += f"\n주소: {row['주소']}"
            result += f"\n현재 위치로부터의 거리: {row['거리']:.2f} km"

            shelter_add = row['주소'].replace(' ', '') 
            kakao_map_link = f"https://map.kakao.com/link/search/{shelter_add}"
            result += f"\n지도 링크: {kakao_map_link}"
        return result
    else:
        return "좌표를 가져올 수 없습니다."


In [18]:
# 함수 스키마 정의
functions = [
    {
        "name": "find_nearest_shelters",
        "description": "주어진 주소를 기반으로 가장 가까운 대피소 정보를 반환하는 함수.",
        "parameters": {
            "type": "object",
            "properties": {
                "address": {
                    "type": "string",
                    "description": "대피소를 찾고자 하는 주소"
                }
            },
            "required": ["address"]
        }
    }
]

In [19]:
# .env 파일 로드
load_dotenv()

# 환경 변수에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")


In [20]:
# 파일 로드
file_path = "비상시국민행동요령.PDF"
# PDF 파일 경로
loader = PyPDFLoader(file_path=file_path)

docs = loader.load()

In [21]:
docs[0:20]

[Document(metadata={'source': '비상시국민행동요령.PDF', 'page': 0}, page_content='비상시 국민행동요령\nCONTENTS\n온 가족이 함께 \n안전하게\n비상사태시 행동요령\n화생방 피해대비 \n행동요령\n인명·시설 피해시 \n행동요령\n비상대비물자 \n준비 및 사용요령\n•만화(웹툰)로 보는  02\n    비상시 국민행동요령\n•비상시 행동요령 08\n•민방공 경보 발령시 행동요령 11\n•일상생활 비상대비 3가지 14\n•화학무기 피해대비 행동요령 20\n•생물학무기 피해대비 행동요령 21 \n•핵·방사능 피폭대비 행동요령 23\n•핵·방사능 피폭대비 생존상식 25\n•대형건물 붕괴·화재시 행동요령 32\n•전기·물·가스 공급 중단시 행동요령 34\n•지하철 피해시 행동요령 35\n•인명·시설 피해복구 행동요령 37\n•비상대비물자 준비요령 40\n•화생방 대비물자 사용요령 42\n•부상자 응급조치 요령 44\n01\n02\n03\n04\n05'),
 Document(metadata={'source': '비상시국민행동요령.PDF', 'page': 1}, page_content='비상시 국민행동요령 알아야 안전하다2\n만화로 보는 \n비상시 국민\n행동요령\n온 가족이 함께 보고,\n           쉽고 재미있게 배워요!\n가정에서, 학교에서, 집 밖에서 언제 어디서나 비상대비!\n남녀노소 누구나 어렵지 않게 비상시 행동요령을 배워보아요.\n* 비상사태 정의 : 전시, 사변이나 이에 준하는 비상 시(비상대비에 관한 법률 제2조)\n엄마가\n불길한 꿈을 \n꾸었대\n비상사태로 다치고,\n우리 가족을 잃어버리는\n꿈을 꾸었다더구나.\n아빠, 엄마\n뭐 하시는 거에요?\n1 경계 경보가 울리면\n이제 끝났다!\n평상시 알고 대비하는게\n가장 중요해.\n당황하지 말고 침착하게 평소 준비해 놓았던 \n                       생활필수품을 확인해야 해요.\n일상생활 중 잘 대비해 놓고\n비상상황에선\n

In [22]:
def preprocess_text(text):
    # 1. 처음에 숫자가 나오면 제거
    text = re.sub(r'^\s*\d+\s*', '', text)
    
    # 2. "비상시 국민행동요령 알아야 안전하다"로 시작하면 제거
    if text.startswith('비상시 국민행동요령 알아야 안전하다'):
        text = text[len('비상시 국민행동요령 알아야 안전하다'):].lstrip()
    
    # 3. 특정 패턴이 시작 부분에 있으면 제거 (패턴 1)
    pattern1 = r'^(\s*만화로 보는 비상시\s*국민행동요령\s*화생방 피해대비\s*행동요령\s*인명시설 피해시\s*행동요령\s*비상대비물자\s*준비 및 사용요령\s*비상사태시\s*행동요령\s*)'
    text = re.sub(pattern1, '', text)
    
    # 4. 특정 패턴이 시작 부분에 있으면 제거 (패턴 2)
    pattern2 = r'^(\s*온 가족이 함께\s*안전하게\s*화생방 피해대비\s*행동요령\s*인명시설 피해시\s*행동요령\s*비상대비물자\s*준비 및 사용요령\s*비상사태시\s*행동요령\s*화생방경보 발령시\s*국민행동요령\s*핵 경보 발령시\s*국민행동요령\s*)'
    text = re.sub(pattern2, '', text)
    
    # 5. 불필요한 줄 바꿈과 공백 정리
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    # 6. 특수 문자 제거 (URL에 사용되는 문자는 제외)
    # URL에 사용되는 문자 ':', '/', '.', '-', '_' 등을 보존합니다.
    text = re.sub(r'[^\w\sㄱ-힣:/\.\-]', '', text)
    # 처리 후 숫자가 나오면 제거
    text = re.sub(r'^\s*\d+\s*', '', text)
    return text


In [23]:
# 각 문서의 page_content 전처리
for doc in docs:
    doc.page_content = preprocess_text(doc.page_content)

In [24]:
docs[0:14]

[Document(metadata={'source': '비상시국민행동요령.PDF', 'page': 0}, page_content='비상시 국민행동요령 CONTENTS 온 가족이 함께 안전하게 비상사태시 행동요령 화생방 피해대비 행동요령 인명시설 피해시 행동요령 비상대비물자 준비 및 사용요령 만화웹툰로 보는 02 비상시 국민행동요령 비상시 행동요령 08 민방공 경보 발령시 행동요령 11 일상생활 비상대비 3가지 14 화학무기 피해대비 행동요령 20 생물학무기 피해대비 행동요령 21 핵방사능 피폭대비 행동요령 23 핵방사능 피폭대비 생존상식 25 대형건물 붕괴화재시 행동요령 32 전기물가스 공급 중단시 행동요령 34 지하철 피해시 행동요령 35 인명시설 피해복구 행동요령 37 비상대비물자 준비요령 40 화생방 대비물자 사용요령 42 부상자 응급조치 요령 44 01 02 03 04 05'),
 Document(metadata={'source': '비상시국민행동요령.PDF', 'page': 1}, page_content='만화로 보는 비상시 국민 행동요령 온 가족이 함께 보고 쉽고 재미있게 배워요 가정에서 학교에서 집 밖에서 언제 어디서나 비상대비 남녀노소 누구나 어렵지 않게 비상시 행동요령을 배워보아요.  비상사태 정의 : 전시 사변이나 이에 준하는 비상 시비상대비에 관한 법률 제2조 엄마가 불길한 꿈을 꾸었대 비상사태로 다치고 우리 가족을 잃어버리는 꿈을 꾸었다더구나. 아빠 엄마 뭐 하시는 거에요 1 경계 경보가 울리면 이제 끝났다 평상시 알고 대비하는게 가장 중요해. 당황하지 말고 침착하게 평소 준비해 놓았던 생활필수품을 확인해야 해요. 일상생활 중 잘 대비해 놓고 비상상황에선 침착하게 대응해야 해. 엄마가 공부한 비상시 행동요령들 알려줄게'),
 Document(metadata={'source': '비상시국민행동요령.PDF', 'page': 2}, page_content='만화로 보는 비상시 국민행동요령 화생방 피해대비 행동요령 인명시설 피해시 행동요령 비상대

In [25]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

# 청크 확인 디버그 코드
for idx, chunk in enumerate(splits):
    print(f"Chunk {idx + 1}:")
    print("-" * 20)
    print(chunk.page_content)
    print("\n" + "=" * 40 + "\n")


Chunk 1:
--------------------
비상시 국민행동요령 CONTENTS 온 가족이 함께 안전하게 비상사태시 행동요령 화생방 피해대비 행동요령 인명시설 피해시 행동요령 비상대비물자 준비 및 사용요령 만화웹툰로 보는 02 비상시 국민행동요령 비상시 행동요령 08 민방공 경보 발령시 행동요령 11 일상생활 비상대비 3가지 14 화학무기 피해대비 행동요령 20 생물학무기 피해대비 행동요령 21 핵방사능 피폭대비 행동요령 23 핵방사능 피폭대비 생존상식 25 대형건물 붕괴화재시 행동요령 32 전기물가스 공급 중단시 행동요령 34 지하철 피해시 행동요령 35 인명시설 피해복구 행동요령 37 비상대비물자 준비요령 40 화생방 대비물자 사용요령 42 부상자 응급조치 요령 44 01 02 03 04 05


Chunk 2:
--------------------
만화로 보는 비상시 국민 행동요령 온 가족이 함께 보고 쉽고 재미있게 배워요 가정에서 학교에서 집 밖에서 언제 어디서나 비상대비 남녀노소 누구나 어렵지 않게 비상시 행동요령을 배워보아요.  비상사태 정의 : 전시 사변이나 이에 준하는 비상 시비상대비에 관한 법률 제2조 엄마가 불길한 꿈을 꾸었대 비상사태로 다치고 우리 가족을 잃어버리는 꿈을 꾸었다더구나. 아빠 엄마 뭐 하시는 거에요 1 경계 경보가 울리면 이제 끝났다 평상시 알고 대비하는게 가장 중요해. 당황하지 말고 침착하게 평소 준비해 놓았던 생활필수품을 확인해야 해요. 일상생활 중 잘 대비해 놓고 비상상황에선 침착하게 대응해야 해. 엄마가 공부한 비상시 행동요령들 알려줄게


Chunk 3:
--------------------
만화로 보는 비상시 국민행동요령 화생방 피해대비 행동요령 인명시설 피해시 행동요령 비상대비물자 준비 및 사용요령 비상사태시 행동요령 2 3 5 6 4 단전으로 조명이 꺼지면 단수에 대비하여 집안의 욕조 양동이 등에 물을 미리 받아 두세요. 비상연락망을 확인 집 밖으로 나가지 말고 정부방송에 귀 기울이며 안내에 따라 행동해

In [26]:
# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


# 로컬 파일 저장소 설정
store = LocalFileStore("F:\STUDY\sparta\999\박성규\emb")

# 캐시를 지원하는 임베딩 생성 - 임베딩시 계속 api 호출을 방지하기 위해 로컬에 임베팅 파일을 저장하는 형식
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [27]:
vectorstore = FAISS.from_documents(documents=splits, embedding=cached_embedder)

In [28]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 가져올 청크 수를 3으로 늘림

## 디버깅

In [29]:
system_prompt = (
    "당신은 비상사태 대처 매뉴얼 챗봇입니다. 제공된 컨텍스트와 일반 지식을 결합하여 질문에 답변하세요. "
    "필요하다면 함수 호출을 사용하세요."
)
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    model_kwargs={"functions": functions}
)

# 사용자 입력 받기
user_input = input("질문을 입력하세요: ")

# 메시지 생성
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_input)
]

# 모델 호출
response = model.invoke(messages)

# 함수 호출 여부 확인
if response.additional_kwargs.get("function_call"):
    function_name = response.additional_kwargs["function_call"]["name"]
    function_args = json.loads(response.additional_kwargs["function_call"]["arguments"])

    # 함수 호출
    if function_name == "find_nearest_shelters":
        function_result = find_nearest_shelters(**function_args)

        # 함수 호출 결과를 모델에 전달하여 최종 답변 생성
        function_message = FunctionMessage(
            name=function_name,
            content=function_result
        )
        messages.append(function_message)
        final_response = model.invoke(messages)
        print(final_response.content)
    else:
        print(f"알 수 없는 함수 호출: {function_name}")
else:
    print(response.content)

KeyboardInterrupt: Interrupted by user

## 함수화해서 디버깅

In [ ]:
# 시스템 프롬프트 정의
system_prompt = (
    "당신은 비상사태 대처 매뉴얼 챗봇입니다. 제공된 컨텍스트와 일반 지식을 결합하여 질문에 답변하세요. "
    "필요하다면 함수 호출을 사용하세요."
)

# 모델 초기화
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    model_kwargs={"functions": functions}
)

# 대화 기록 초기화
conversation_history = [SystemMessage(content=system_prompt)]

def get_user_input():
    return input("사용자: ")

def process_response(response):
    if response.additional_kwargs.get("function_call"):
        function_name = response.additional_kwargs["function_call"]["name"]
        function_args = json.loads(response.additional_kwargs["function_call"]["arguments"])

        # 함수 호출 처리
        if function_name == "find_nearest_shelters":
            function_result = find_nearest_shelters(**function_args)
            function_message = FunctionMessage(
                name=function_name,
                content=function_result
            )
            conversation_history.append(function_message)
            final_response = model.invoke(conversation_history)
            print(f"챗봇: {final_response.content}")
            conversation_history.append(AIMessage(content=final_response.content))
        else:
            print(f"알 수 없는 함수 호출: {function_name}")
    else:
        print(f"챗봇: {response.content}")
        conversation_history.append(AIMessage(content=response.content))

def main():
    while True:
        user_input = get_user_input()
        if user_input.lower() in ["종료", "exit", "quit"]:
            print("대화를 종료합니다.")
            break
        user_message = HumanMessage(content=user_input)
        conversation_history.append(user_message)
        response = model.invoke(conversation_history)
        process_response(response)

if __name__ == "__main__":
    main()

### 추가해야할 점

1. 프롬프트를 수정해서 원하는 답변유도를 열심히 수정해야 합니다.
2. 체인형성으로 좀더 가독성 있는 코드로 수정해야 합니다.
3. 추가적으로 RAG 활용할 파일들을 확보해야 합니다.
4. 여러가지 입력을 통해서 예외처리를 해야합니다.

## 체인구조 구성하기

In [32]:
# 시스템 프롬프트 정의
system_prompt = (
    "당신은 비상사태 대처 매뉴얼 챗봇입니다. 제공된 컨텍스트와 일반 지식을 결합하여 질문에 답변하세요. "
    "필요하다면 함수 호출을 사용하세요."
)

conversation_history = [SystemMessage(content=system_prompt)]

def get_user_input():
    return input("사용자: ")

def handle_function_call(response):
    function_name = response.additional_kwargs["function_call"]["name"]
    function_args = json.loads(response.additional_kwargs["function_call"]["arguments"])

    if function_name == "find_nearest_shelters":
        function_result = find_nearest_shelters(**function_args)
        function_message = FunctionMessage(
            name=function_name,
            content=function_result
        )
        conversation_history.append(function_message)
        final_response = model.invoke(conversation_history)
        print(f"챗봇: {final_response.content}")
        conversation_history.append(AIMessage(content=final_response.content))
    else:
        print(f"알 수 없는 함수 호출: {function_name}")

def process_response(response):
    if response.additional_kwargs.get("function_call"):
        handle_function_call(response)
    else:
        print(f"챗봇: {response.content}")
        conversation_history.append(AIMessage(content=response.content))


# 사용자 입력을 받아 HumanMessage로 변환하는 Runnable
class UserInputToMessage(Runnable):
    def invoke(self, input, config=None):
        return HumanMessage(content=input)

# 대화 기록에 메시지를 추가하는 Runnable
class AddMessageToHistory(Runnable):
    def invoke(self, message, config=None):
        conversation_history.append(message)
        return conversation_history

# 모델을 호출하는 Runnable
class ModelInvocation(Runnable):
    def invoke(self, history, config=None):
        return model.invoke(history)

# 응답을 처리하는 Runnable
class ResponseProcessor(Runnable):
    def invoke(self, response, config=None):
        process_response(response)
        return response

# 체인 구성
chat_chain = (
    UserInputToMessage()
    | AddMessageToHistory()
    | ModelInvocation()
    | ResponseProcessor()
)

def main():
    while True:
        user_input = get_user_input()
        if user_input.lower() in ["종료", "exit", "quit"]:
            print("대화를 종료합니다.")
            break
        chat_chain.invoke(user_input)

if __name__ == "__main__":
    main()



사용자:  서울시청 근처 대피소


C:\Users\1\AppData\Local\Temp\ipykernel_13704\2483391231.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['거리'] = df_result.apply(calculate_distance, axis=1)


챗봇: 서울시청 근처의 대피소는 다음과 같습니다:

1. **덕수궁앞지하보도**
   - 주소: 서울특별시 중구 세종대로 124 (태평로1가, 프레스센터)
   - 거리: 0.11 km
   - [지도 링크](https://map.kakao.com/link/search/서울특별시중구세종대로124(태평로1가,프레스센터))

2. **서울특별시청 지하1~4층**
   - 주소: 서울특별시 중구 세종대로 110 (태평로1가, 서울시청)
   - 거리: 0.11 km
   - [지도 링크](https://map.kakao.com/link/search/서울특별시중구세종대로110(태평로1가,서울시청))

3. **시청광장지하쇼핑센터**
   - 주소: 서울특별시 중구 을지로 지하 12 (을지로1가, 시청광장지하쇼핑센터)
   - 거리: 0.14 km
   - [지도 링크](https://map.kakao.com/link/search/서울특별시중구을지로지하12(을지로1가,시청광장지하쇼핑센터))

이 대피소들은 서울시청에서 매우 가까운 거리에 위치해 있습니다. 필요 시 해당 대피소로 이동하시기 바랍니다.


사용자:  서울 중구 세종대로 110 근처 대피소


C:\Users\1\AppData\Local\Temp\ipykernel_13704\2483391231.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['거리'] = df_result.apply(calculate_distance, axis=1)


챗봇: 서울 중구 세종대로 110 근처의 대피소는 다음과 같습니다:

1. **서울특별시청 지하1~4층**
   - 주소: 서울특별시 중구 세종대로 110 (태평로1가, 서울시청)
   - 거리: 0.02 km
   - [지도 링크](https://map.kakao.com/link/search/서울특별시중구세종대로110(태평로1가,서울시청))

2. **1호선 시청역 지하철 승강장**
   - 주소: 서울특별시 중구 세종대로 지하 101 (정동, 시청역 1호선)
   - 거리: 0.12 km
   - [지도 링크](https://map.kakao.com/link/search/서울특별시중구세종대로지하101(정동,시청역1호선))

3. **시청광장지하쇼핑센터**
   - 주소: 서울특별시 중구 을지로 지하 12 (을지로1가, 시청광장지하쇼핑센터)
   - 거리: 0.13 km
   - [지도 링크](https://map.kakao.com/link/search/서울특별시중구을지로지하12(을지로1가,시청광장지하쇼핑센터))

이 대피소들은 세종대로 110에서 매우 가까운 거리에 위치해 있습니다. 필요 시 해당 대피소로 이동하시기 바랍니다.


사용자:  종료


대화를 종료합니다.


추가기능 : 스트리밍 !!